In [7]:
import numpy as np
import pandas as pd

In [9]:
movies=pd.read_csv(r"C:\Users\muri-\Desktop\datasets\movies.csv")

In [10]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [61]:
movies = movies[['movieId','title','genres']]
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [28]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [29]:
movies.duplicated().sum()

0

In [30]:
movies.dropna(inplace=True)

In [88]:
movies.iloc[1].genres

'Adventure|Children|Fantasy'

In [85]:
import ast
ast.literal_eval("[{'name': 'Animation'}, { 'name': 'Adventure'}, { 'name': 'Fantasy'}, { 'name': 'Comedy'},{'name':'Children'}]")

[{'name': 'Animation'},
 {'name': 'Adventure'},
 {'name': 'Fantasy'},
 {'name': 'Comedy'},
 {'name': 'Children'}]

In [86]:
def convert(obj):
    L = []
    for i in ast.literal_eval(str(obj)):
        L.append(i['name'])
    return L

In [ ]:
movies["genres"].apply(convert)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(1)

In [ ]:
def convert3(obj):
    L = []
    c=0
    for i in ast.literal_eval(obj):
        if(c!=3):
            L.append(i['name'])
            c+=1
        else:
            break
    return L
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors

In [ ]:
cv.get_feature_names()

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
ps.stem('called')
#output:
call

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors)

In [ ]:
similarity = cosine_similarity(vectors).shape


In [ ]:
similarity[0]

In [ ]:

movie_index = new_df[new_df['title']==movie].index[0]
distances = similarity[movie_index]
movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])

In [ ]:
def önerilen(movie):

    #find the index of the movies
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    

    #to fetch movies from indeces
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
import  streamlit as st
st.title('Movie Recommender System')

In [ ]:
import pickle
pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))

In [ ]:
import streamlit as st
import pickle
import pandas as pd

movies_dict = pickle.load(open('movies.pkl','rb'))
movies = pd.DataFrame(movies_dict)

st.title('Movie Recommender System')

selected_movie_name = st.selectbox(
"Type or select a movie from the dropdown",
 movies['title'].values
)

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
similarity = pickle.load(open('similarity.pkl','rb'))
def önerilen(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = []
    for i in movies_list:
             recommended_movies.append(movies.iloc[i[0]].title)
    return recommended_movies

if st.button('Show Recommendation'):
    recommendations = recommend(selected_movie_name)

    for i in recommendations:
        st.write(i)

In [ ]:
import requests

def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key='Your API Key'&language=en-US".format(movie_id)
    data = requests.get(url)
    data = data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path

In [ ]:
import  streamlit as st

import pickle
import pandas as pd
import requests

st.title('Movie Recommender System')


def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key='YOUR API KEY'&language=en-US".format(movie_id)
    data = requests.get(url)
    data = data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path

def önerilen(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = []
    recommended_movie_posters = []

    for i in movies_list:
        # fetch the movie poster
        movie_id = movies.iloc[i[0]].id
        recommended_movies.append(movies.iloc[i[0]].title)
        recommended_movie_posters.append(fetch_poster(movie_id))

    return recommended_movies,recommended_movie_posters

movies_dict = pickle.load(open('movies.pkl','rb'))
movies = pd.DataFrame(movies_dict)
similarity = pickle.load(open('similarity.pkl','rb'))


selected_movie_name = st.selectbox(
    "Type or select a movie from the dropdown",
    movies['title'].values
)
if st.button('önerilenleri göster'):
    names,posters = önerilen(selected_movie_name)

    #display with the columns
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        st.text(names[0])
        st.image(posters[0])
    with col2:
        st.text(names[1])
        st.image(posters[1])
    with col3:
        st.text(names[2])
        st.image(posters[2])
    with col4:
        st.text(names[3])
        st.image(posters[3])
    with col5:
        st.text(names[4])
        st.image(posters[4])